apply 메서드는 사용자가 작성한 함수를 한 번에 df의 각 행과 열에 적용하여
실행할 수 있게 해줌. 즉, 함수를 브로드캐스팅해야 하는 경우에 사용.

물론, for문으로 각 데이터에 함수를 적용할 수 있지만 대용량 데이터 처리시
apply 메서드가 유리함.

# 제곱 함수와 n 제곱 함수 만들기

In [1]:
def my_sq(x):
    return x ** 2

def my_exp(x,n):
    return x ** n

print(my_sq(4))
print(my_exp(3,3))

16
27


# 시리즈와 apply 메서드

In [5]:
# apply로 브로드캐스팅한 결과와 s, df의 기초 연산 결과와 비교하며 진행

# 먼저 df 준비
import pandas as pd
df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
print(df)

# 두 결과 비교
print(df['a']**2)

sq = df['a'].apply(my_sq) # apply 메서드에 함수 이름 전달. 이게 곧 x
print(sq)

# 두개의 인자를 apply에 전달하는 경우
ex = df['a'].apply(my_exp, n=2) 
print(ex)



    a   b
0  10  20
1  20  30
2  30  40
0    100
1    400
2    900
Name: a, dtype: int64
0    100
1    400
2    900
Name: a, dtype: int64
0    100
1    400
2    900
Name: a, dtype: int64


# 데이터 프레임과 apply 메서드

In [15]:
df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
print(df)

def print_me(x):
    print(x)

print(df.apply(print_me, axis=0))
print(df['a'])
print(df['b'])

# 3개의 인자를 받는 함수
def avg_3(x,y,z):
    return (x+y+z)/3

# print(df.apply(avg_3)) 실행 시 오류가 난다. why? 1개의 인잣값만 받아서
# 즉, df['a'] 등이 전달되었고 이 값을 1개의 인잣값으로 인식했으니
# avg_3 함수가 열 단위로 데이터를 처리할 수 있게 수정해야함
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return(x+y+z)/3
print(df.apply(avg_3_apply))

    a   b
0  10  20
1  20  30
2  30  40
0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object
0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    20.0
b    30.0
dtype: float64


In [16]:
# 앞의 과정에서는 df의 행 개수가 3이라는 걸 알고 있으니 가능한 과정
# 하지만 일반적으로는 for문을 이용하여 아래와 같이 작성한다

def avg_3_apply(col):
    sum = 0
    for item in col:
        sum += item
    return sum/df.shape[0]

# 위 함수를 응용하면 행 방향으로 데이터를 처리하는 함수 제작 가능
def avg_2_apply(row):
    sum = 0
    for item in row:
        sum += item
    return sum/df.shape[1]
print(df.apply(avg_2_apply, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


# 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기 - 열 방향

In [22]:
# seaborn 라이브러리에 있는 데이터 사용
import seaborn as sns
titanic = sns.load_dataset("titanic")

print(titanic.info())


# 누락값의 개수를 반환하는 count_missing 함수
# isnull 메서드에 df 전달 후 T/F 출력
# 이 값을 numpy의 sum 메서드에 전달하면 누락값 개수 출력
import numpy as np

def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count


cmis_col = titanic.apply(count_missing) # apply 메서드에 함수 전달
print(cmis_col)


# 누락값의 비율을 계산하는 prop_missing 함수
# size 속성을 이용해 df의 전체 데이터 수를 구해 나누면 비율이 돼

def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

pmis_col = titanic.apply(prop_missing) # apply 메서드에 함수 전달
print(pmis_col)

# 전체 비율에서 누락값의 비율을 빼면 누락값이 아닌 데이터의 비율
def prop_complete(vec):
    return 1 - prop_missing(vec)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None
survived         0
pclass           0
sex              0
age            17

# 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기 - 행 방뱡

In [26]:
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)

print(cmis_row.head())
print(pmis_row.head())
print(pcom_row.head())

# 누락값의 개수를 구하여 titanic df에 'num_missing' 열로 추가
titanic['num_missing'] = titanic.apply(count_missing, axis=1)
print(titanic.head())

# 추가한 열에서 누락값이 2개 이상인 데이터만 추출하기
print(titanic.loc[titanic.num_missing > 1, :])

0    1
1    0
2    1
3    0
4    1
dtype: int64
0    0.0625
1    0.0000
2    0.0625
3    0.0000
4    0.0625
dtype: float64
0    0.9375
1    1.0000
2    0.9375
3    1.0000
4    0.9375
dtype: float64
   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man  